In [1]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [2]:
limefeat_alphasort = pd.read_csv("limefeat_alphasort.csv")
officer_table = pd.read_csv("officer_table.csv")

In [3]:
def relister(liststring):
    l = []
    tmp = liststring.lstrip("[").rstrip("]")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [4]:
officer_table['feats'] = officer_table.feats.apply(relister)

#### Feature categories

In [5]:
cathue = {
    'ts':'0',
    'fi':'32',
    'ir':'60',
    'shifts':'115',
    'dispatch':'180',
    'arrests':'240',
    'ic':'270',
    'demarrests':'300',
    'ocnd':'200'
    }

catsat = {
    'ts':'100%',
    'fi':'100%',
    'ir':'100%',
    'shifts':'100%',
    'dispatch':'100%',
    'arrests':'100%',
    'ic':'100%',
    'demarrests':'100%',
    'ocnd':'100%'
    }

catval = {
    '1d':'5%',
    '1w':'25%',
    '1m':'50%',
    '1y':'75%',
    '5y':'100%',
    'all':'100%',
    }

In [6]:
officer_table.sort_values(by="unit_score",ascending=False,inplace=True)
officer_table.reset_index(drop=True,inplace=True)

In [7]:
rectwidth = 16
rectheight = 16
px_w = len(limefeat_alphasort) * rectwidth
px_h = len(officer_table) * rectheight

In [8]:
def offtable(officer_table,feature_table):
    canvas = Image.new('RGB',(px_w,px_h),'hsl(0,0%,100%)')
    for i in officer_table.index:
        feats = officer_table.feats.loc[i]
        ycoord = i * rectheight
        #outcome = officer_table.outcome.loc[i]
        
        for feat in feats:
            tmp = feature_table[feature_table.feature==feat]
            record_type = tmp.record_type.iloc[0]
            idx = tmp.index[0]
            
            xcoord = idx * rectwidth
            bbox = [xcoord, ycoord, xcoord + rectwidth, ycoord + rectheight]
            
            try:
                hue = cathue[record_type]
                sat = catsat[record_type]
                val = '50%'
            except:
                hue = '0'
                sat = '0%'
                val = '100%'

            fillcolor = 'hsl('+hue+","+sat+","+val+")"
            draw = ImageDraw.Draw(canvas)
            draw.rectangle(bbox,fill=fillcolor,outline=None)
    return canvas

In [9]:
offtable(officer_table,limefeat_alphasort).save("officer_table.png")